In [1]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [3]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame

# batteries
battery = CSV.File("../data/Battery.csv") |> DataFrame

# transmission_lines
transmission_lines = CSV.File("../data/transmission_lines.csv") |> DataFrame


Row,Transmission lines: From node,To node,Susceptance [per-unit],Capacity [MW]
,Int64,Int64,Float64,Int64
1,1,2,0.0146,175
2,1,3,0.2253,175
3,1,5,0.0907,350
4,2,4,0.1356,175
5,2,6,0.205,175
6,3,9,0.1271,175
7,3,24,0.084,400
8,4,9,0.111,175
9,5,10,0.094,350


In [4]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

# number of batteries
B = size(battery, 1)

# number of transmission lines
L = size(transmission_lines, 1)

# number of nodes
N = 24


24

In [5]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    result_df[!, Symbol("x_bat$i")] = Float64[]
end

# For l variables, manually add each column. This is a one-time setup.
for i in 1:L
    result_df[!, Symbol("x_line$i")] = Float64[]
end



equilibrium_df = DataFrame(hour = Int[])

# for each node a market price column
for i in 1:N
    equilibrium_df[!, Symbol("market_price_node$i")] = Float64[]
end



In [6]:
println(result_df)

0×73 DataFrame
 Row │ hour   x_con1   x_con2   x_con3   x_con4   x_con5   x_con6   x_con7   x_con8   x_con9   x_con10  x_con11  x_con12  x_wind1  x_wind2  x_wind3  x_wind4  y1       y2       y3       y4       y5       y6       y7       y8       y9       y10      y11      y12      y13      y14      y15      y16      y17      x_bat1   x_bat2   x_bat3   x_bat4   x_bat5   x_line1  x_line2  x_line3  x_line4  x_line5  x_line6  x_line7  x_line8  x_line9  x_line10  x_line11  x_line12  x_line13  x_line14  x_line15  x_line16  x_line17  x_line18  x_line19  x_line20  x_line21  x_line22  x_line23  x_line24  x_line25  x_line26  x_line27  x_line28  x_line29  x_line30  x_line31  x_line32  x_line33  x_line34 
     │ Int64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float

In [7]:
# Load demand bids data
demand_bids_all = [CSV.File("../data/demand_bids_hour/demand_hour_$(i-1).csv") |> DataFrame for i in 1:24]

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator and hour
@variable(model, x_con[1:G, 1:24] >= 0)  # Power output variable for conventional generators
@variable(model, x_wind[1:W, 1:24] >= 0)  # Power output variable for wind generators
@variable(model, y[1:D, 1:24] >= 0)  # Demand variable
@variable(model, bat[1:B, 1:24])  # Battery variable
@variable(model, l[1:L, 1:24])  # Transmission line variable


# Add constraints for each plant
for g in 1:G
    for hour in 1:24
        @constraint(model, x_con[g, hour] <= con_generation[g, 6])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each wind farm
for w in 1:W
    for hour in 1:24
        @constraint(model, x_wind[w, hour] <= wind_profile[hour, w+1])  # Set the upper bound
        @constraint(model, x_wind[w, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each demand bid
for d in 1:D
    for hour in 1:24
        @constraint(model, y[d, hour] <= demand_bids_all[hour][d, 2])  # Set the upper bound
        @constraint(model, y[d, hour] >= 0)  # Set the lower bound
    end
end

# Add temporary for each generator
for g in 1:G
    for hour in 2:24
        @constraint(model, x_con[g, hour] <= x_con[g, hour - 1] + con_generation[g, 7])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= x_con[g, hour - 1] - con_generation[g, 8])  # Set the lower bound
    end
end

# add constraints for battery charging and discharging
for b in 1:B
    for hour in 1:24
        @constraint(model, bat[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat[b, hour] >= -1 *battery[b, 5])  # Set the lower bound
    end
end

# add constraints for battery capacity
# sum all previous battery charging and discharging, they need to be below the battery capacity
for b in 1:B
    for hour in 2:24
        @constraint(model, sum(bat[b, h] for h in 1:hour) <= battery[b, 4])  # Set the upper bound
        @constraint(model, sum(bat[b, h] for h in 1:hour) >= 0)  # Set the lower bound
    end
end

# add constraints for transmission lines
for line in 1:L
    for hour in 1:24
        @constraint(model, l[line, hour] <= transmission_lines[line, 4] * 0.5)  # Set the upper bound  #half the cap to set the different prices per zone
        @constraint(model, l[line, hour] >= -1 * transmission_lines[line, 4] * 0.5)  # Set the lower bound
    end
end

balance = Vector{Any}(undef, N*24)

# add balance constraint for each node and each demand_hour_
for hour in 1:24
    for node in 1:N
        balance[(hour-1)*24+node] = @constraint(model, sum(x_con[g, hour] for g in 1:G if con_generation[g, 2] == node) +
                            sum(x_wind[w, hour] for w in 1:W if wind_generation[w, 2] == node)  +
                            sum(l[line, hour] for line in 1:L if transmission_lines[line, 2] == node) 
                            ==
                            sum(y[d, hour] for d in 1:D if demand_bids_all[hour][d, 1] == node) +
                            sum(l[line, hour] for line in 1:L if transmission_lines[line, 1] == node) +
                            sum(bat[b, hour] for b in 1:B if battery[b, 2] == node) )
    end
end

# Define the objective function
@objective(model, Max, sum(demand_bids_all[hour][d, 3] * y[d, hour] for d in 1:D, hour in 1:24) - sum(con_generation[g, 3] * x_con[g, hour] for g in 1:G, hour in 1:24) )


# Solve the model
optimize!(model)


# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")

    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))


else
    println("No optimal solution found")
end


Optimal solution found
Objective value: 605155.5396795765


In [8]:
# print decision variables
empty!(result_df)
for hour in 1:24
    resultvector = zeros(1+G+W+D+B+L)
    resultvector[1] = hour
    for g in 1:G
        resultvector[1+g] = value(x_con[g, hour])
    end
    for w in 1:W
        resultvector[1+G+w] = value(x_wind[w, hour])
    end
    for d in 1:D
        resultvector[1+G+W+d] = value(y[d, hour])
    end
    for b in 1:B
        resultvector[1+G+W+D+b] = value(bat[b, hour])
    end
    for line in 1:L
        resultvector[1+G+W+D+B+line] = value(l[line, hour])
    end
    push!(result_df, resultvector)
end

In [9]:
#save the duals of the balance constraints to the equilibrium_df
empty!(equilibrium_df)
for hour in 1:24
    hour_results = []

    for node in 1:N
        push!(hour_results, dual(balance[(hour-1)*24+node]))
    end
    push!(equilibrium_df, [hour; hour_results...])
end




#save results to dataframe
CSV.write("results/optimization_results_marketprices_nodal_half.csv", equilibrium_df)



"results/optimization_results_marketprices_nodal_half.csv"

In [10]:
equilibrium_df

Row,hour,market_price_node1,market_price_node2,market_price_node3,market_price_node4,market_price_node5,market_price_node6,market_price_node7,market_price_node8,market_price_node9,market_price_node10,market_price_node11,market_price_node12,market_price_node13,market_price_node14,market_price_node15,market_price_node16,market_price_node17,market_price_node18,market_price_node19,market_price_node20,market_price_node21,market_price_node22,market_price_node23,market_price_node24
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,13.32,13.32,13.32,13.32,13.32,13.32,20.7,20.7,13.32,13.32,13.32,13.32,13.32,13.32,11.075,11.075,6.02,6.02,11.075,11.075,6.02,6.02,11.075,11.075
2,2,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,11.075,11.075,6.02,6.02,11.075,11.075,6.02,6.02,11.075,11.075
3,3,13.32,13.32,10.52,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,10.52,10.52,6.02,6.02,10.52,10.52,6.02,6.02,10.52,10.52
4,4,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,10.52,10.52,6.02,6.02,10.89,10.89,6.02,6.02,10.89,10.52
5,5,13.32,13.32,10.52,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,10.52,10.52,6.02,6.02,10.89,10.89,6.02,6.02,10.89,10.52
6,6,13.32,13.32,10.52,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,13.32,10.52,10.52,6.02,6.02,10.52,10.52,6.02,6.02,10.52,10.52
7,7,15.3136,16.0763,10.52,16.0763,15.3136,16.0763,16.0763,16.0763,16.0763,16.0763,16.0763,16.0763,16.0763,16.0763,10.52,10.52,6.02,6.02,10.52,10.52,6.02,6.02,10.52,10.52
8,8,16.549,16.549,11.63,16.549,16.549,16.549,20.7,20.7,16.549,16.549,16.549,16.549,16.549,16.549,11.63,11.63,6.02,6.02,11.63,11.63,6.02,6.02,11.63,11.63
9,9,16.549,16.549,14.1967,16.549,16.549,16.549,16.8214,16.8214,16.549,16.549,16.549,16.549,16.549,16.549,14.1967,14.1967,6.02,6.02,14.1967,14.1967,6.02,6.02,14.1967,14.1967


In [11]:
result_df

Row,hour,x_con1,x_con2,x_con3,x_con4,x_con5,x_con6,x_con7,x_con8,x_con9,x_con10,x_con11,x_con12,x_wind1,x_wind2,x_wind3,x_wind4,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,x_bat1,x_bat2,x_bat3,x_bat4,x_bat5,x_line1,x_line2,x_line3,x_line4,x_line5,x_line6,x_line7,x_line8,x_line9,x_line10,x_line11,x_line12,x_line13,x_line14,x_line15,x_line16,x_line17,x_line18,x_line19,x_line20,x_line21,x_line22,x_line23,x_line24,x_line25,x_line26,x_line27,x_line28,x_line29,x_line30,x_line31,x_line32,x_line33,x_line34
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,89.9314,51.4173,9.68684,0.0,0.0,108.5,108.5,277.402,280.0,0.0,217.0,178.337,76.8921,101.54,92.8003,95.3709,67.4817,60.3784,165.153,4.26326e-14,197.118,62.1542,207.773,113.653,79.9126,111.878,46.1717,44.3959,85.2401,78.1367,106.55,108.326,120.757,15.0,10.0,10.0,2.29888,-5.0,67.8729,-77.5145,32.0912,46.1717,12.7401,87.5,-200.0,0.0,89.2354,-87.5,-68.4499,-87.5,-87.5,0.0,-108.326,-84.8473,-131.674,40.1527,-125.0,0.0,-250.0,-125.0,-125.0,-90.9166,-200.0,200.0,-250.0,173.23,-69.6291,-175.371,0.0,59.5761,-20.3365,175.371
2,2,106.4,99.4173,0.0,0.0,0.0,108.5,108.5,262.431,280.0,0.0,217.0,130.337,66.8277,90.9989,109.167,107.937,63.453,56.7737,155.293,113.547,185.349,58.4435,195.368,106.868,75.1417,105.198,43.4152,41.7454,80.1511,73.4719,100.189,101.859,113.547,15.0,0.0,0.0,-2.29888,5.0,0.0,-82.7994,125.746,0.0,42.6435,78.8299,-200.0,-43.4152,175.0,-52.5076,-73.4719,-86.1608,-87.5,-102.605,-50.0,121.445,-200.0,30.2928,-11.4526,0.0,-250.0,-125.0,-125.0,-74.5506,-200.0,200.0,-250.0,209.673,-250.0,-5.0,-182.937,102.805,27.6635,5.0
3,3,90.1432,51.4173,0.0,0.0,0.0,108.5,108.5,226.285,280.0,0.0,217.0,82.3365,78.422,116.959,142.88,134.78,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,10.0,10.0,6.92328,5.0,87.5,-87.5,29.7118,-2.65295,87.5,87.5,-196.767,-44.0007,106.913,-3.8344,-69.9732,-77.8912,-87.5,-114.298,-17.1021,97.4385,-200.0,0.0,-16.8596,22.8979,-250.0,-125.0,-125.0,-71.7134,-200.0,196.767,-250.0,249.006,-250.0,-5.0,-209.78,147.227,75.6635,5.0
4,4,58.4,60.271,0.0,0.0,0.0,105.244,108.5,224.3,280.0,0.0,217.0,78.4537,64.1437,134.249,159.57,133.664,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,10.0,10.0,20.0,5.0,87.5,-78.1246,-10.3996,7.10194,87.5,87.5,-200.0,-33.5567,84.7541,-2.56216,-68.807,-75.1347,-87.5,-116.583,0.0,78.3539,-200.0,-19.5671,-18.6619,40.0,-250.0,-125.0,-125.0,-88.3374,-200.0,200.0,-250.0,250.0,-250.0,-5.0,-208.664,149.917,79.5463,5.0
5,5,10.4,87.6367,0.0,0.0,0.0,108.5,75.1699,186.946,280.0,0.0,217.0,78.4537,102.22,145.507,160.944,165.303,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,10.0,10.0,20.0,-0.714959,87.5,-87.5,-49.0242,34.4677,87.5,87.5,-171.3,-6.191,57.3884,-2.56216,-68.807,-87.5,-75.1347,98.4175,-200.0,-136.647,0.0,-19.5671,-18.6619,40.0,-250.0,-125.0,-125.0,-56.3808,-200.0,171.3,-250.0,250.0,-3.98178,-245.303,0.0,149.917,79.5463,245.303
6,6,58.4,69.007,0.0,0.0,0.0,72.5,48.7645,199.238,280.0,0.0,217.0,121.389,134.039,131.112,157.023,161.828,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,10.0,10.0,20.0,5.0,72.5632,-87.5,12.9054,0.0,87.5,87.5,-141.15,-41.3478,104.26,-3.

In [12]:
#save result_df to csv
CSV.write("results/market_clearing_nodal_half.csv", result_df)

"results/market_clearing_nodal_half.csv"